<a href="https://colab.research.google.com/github/sonalshreya25/DeepLearning/blob/main/Assignment4_CNN_multi_class_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
from torch.utils.data import DataLoader, random_split
import numpy as np
import kagglehub

# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)


In [2]:

# Initialize Weights & Biases
wandb.init(project='CNN', entity='usf-alpha-neurons', config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "FashionMNIST",
        "epochs": 10,
    },)

# Simulate training.
epochs = 10

# Download dataset using kagglehub
dataset_path = kagglehub.dataset_download("zalando-research/fashionmnist")

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
full_trainset = torchvision.datasets.FashionMNIST(root=dataset_path, train=True, download=True, transform=transform)
train_size = int(0.8 * len(full_trainset))
val_size = len(full_trainset) - train_size
trainset, valset = random_split(full_trainset, [train_size, val_size])

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=False)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sonalshreya (usf-alpha-neurons) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.68MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 7.97MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [3]:

# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x



In [4]:
# Model, Loss, and Optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:

# Training loop
def train_model(model, trainloader, valloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        val_loss, val_accuracy = evaluate_model(model, valloader)
        wandb.log({'epoch': epoch+1, 'loss': avg_loss, 'val_loss': val_loss, 'val_accuracy': val_accuracy})
        print(f'Epoch {epoch+1}, Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')



In [7]:
# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = running_loss / len(dataloader)
    return avg_loss, accuracy



In [8]:
# Run training and evaluation
train_model(model, trainloader, valloader, criterion, optimizer, epochs=10)
test_loss, test_accuracy = evaluate_model(model, testloader)
wandb.log({'Test Loss': test_loss, 'Test Accuracy': test_accuracy})
print(f'Accuracy on test set: {test_accuracy:.2f}%')


Epoch 1, Loss: 0.5804, Val Loss: 0.3777, Val Accuracy: 86.28%
Epoch 2, Loss: 0.3015, Val Loss: 0.2802, Val Accuracy: 89.87%
Epoch 3, Loss: 0.2513, Val Loss: 0.2623, Val Accuracy: 90.68%
Epoch 4, Loss: 0.2213, Val Loss: 0.2570, Val Accuracy: 90.71%
Epoch 5, Loss: 0.1940, Val Loss: 0.2361, Val Accuracy: 91.47%
Epoch 6, Loss: 0.1728, Val Loss: 0.2373, Val Accuracy: 91.84%
Epoch 7, Loss: 0.1514, Val Loss: 0.2339, Val Accuracy: 92.01%
Epoch 8, Loss: 0.1298, Val Loss: 0.2296, Val Accuracy: 91.92%
Epoch 9, Loss: 0.1103, Val Loss: 0.2570, Val Accuracy: 92.11%
Epoch 10, Loss: 0.0928, Val Loss: 0.2673, Val Accuracy: 91.88%
Accuracy on test set: 91.12%
